In [33]:
import MaterialX as mx

print("MaterialX version:", mx.getVersionString())

def createTestDoc():
    doc : mx.Document = mx.createDocument()

    stdlib = mx.createDocument()
    libFiles = mx.loadLibraries(mx.getDefaultDataLibraryFolders(), mx.getDefaultDataSearchPath(), stdlib)  

    defs = stdlib.getNodeDefs()
    print("Number of node definitions loaded:", len(defs))
    return doc, stdlib

doc, stdlib = createTestDoc()

MaterialX version: 1.39.5
Number of node definitions loaded: 803


In [34]:



def test_make_functional_definition(test_name):
    test_def = stdlib.getNodeDef(test_name)
    if test_def:
        print(f"Node Definition '{test_name}' found.")
        #print(mx.prettyPrint(test_def))

        graph = test_def.getImplementation()
        if graph:
            if graph.isA(mx.NodeGraph) :
                #print(mx.prettyPrint(graph))
                newGraph = test_def.makeFunctionalDefinition()
            
                if False and not newGraph:
                    # This is the C++ code in Python form
                    ngname = graph.getNodeDefString()
                    qualname = graph.getQualifiedName(test_def.getName())
                    print("nodedef string:", ngname)
                    print("qualified name:", qualname)
                    if ngname == qualname:
                        newGraphName = graph.getName()
                        newGraph = test_def.addChildOfCategory("nodegraph", newGraphName)
                        if not newGraph:
                            print("Failed to create new functional node graph:", newGraphName)
                        else:   
                            print("Created new functional node graph:", newGraphName)
                            newGraph.copyContentFrom(graph)
                            newGraph.removeAttribute(mx.InterfaceElement.NODE_DEF_ATTRIBUTE)
                            
                            parent = test_def.getParent()
                            graph.removeAttribute(mx.InterfaceElement.NODE_DEF_ATTRIBUTE)
                            #tempName = parent.createValidChildName(newGraphName + "_old")
                            #graph.setName(tempName)
                        

                if newGraph:
                    print("New functional node definition name:", newGraph.getName())
                    print(mx.prettyPrint(test_def))

test_name = "ND_tiledimage_color3"
test_make_functional_definition(test_name)

test_name = "ND_tiledimage_color4"
test_make_functional_definition(test_name)

Node Definition 'ND_tiledimage_color3' found.
New functional node definition name: NG_tiledimage_color3
<nodedef name="ND_tiledimage_color3" node="tiledimage" nodegroup="texture2d">
  <input name="file" type="filename" value="" uniform="true">
  <input name="default" type="color3" value="0.0, 0.0, 0.0">
  <input name="texcoord" type="vector2" defaultgeomprop="UV0">
  <input name="uvtiling" type="vector2" value="1.0, 1.0">
  <input name="uvoffset" type="vector2" value="0.0, 0.0">
  <input name="realworldimagesize" type="vector2" value="1.0, 1.0" unittype="distance">
  <input name="realworldtilesize" type="vector2" value="1.0, 1.0" unittype="distance">
  <input name="filtertype" type="string" value="linear" enum="closest,linear,cubic" uniform="true">
  <input name="framerange" type="string" value="" uniform="true">
  <input name="frameoffset" type="integer" value="0" uniform="true">
  <input name="frameendaction" type="string" value="constant" enum="constant,clamp,periodic,mirror" unifor

In [ ]:

def get_matching_definitions(def_name):
    stdsurf = stdlib.getNodeDef(def_name)
    nodegraph_counts = {}  # Will store {nodegraph: count}
    nodegraph_nodedefs = {}  # Will store {nodegraph: set(nodedefs)}
    if stdsurf:
        print(f"got node def: {stdsurf.getVersionString()}")

        other_stdsurf = stdsurf.getMatchingDefinitions()
        print("* number of matching definitions:", len(other_stdsurf))
        for ndstring in other_stdsurf:
            print("matching definition:", ndstring)
            nd = stdlib.getNodeDef(ndstring)
            if nd:
                mapped_other_stdsurf = stdlib.getMatchingIndirectImplementations(ndstring)
                print("number of mapped implementations:", len(mapped_other_stdsurf))
                for impl in mapped_other_stdsurf:
                    print("- mapping implementations:", impl.getName())

                print("  version:", nd.getVersionString(), " inherits from:", nd.getInheritString())
                impl = nd.getImplementation()
                if impl.isA(mx.NodeGraph):
                    print("  nodegraph implementation:", impl.getName()) 
                else:           
                    nodegraph_name = None
                    nodegraph_string = impl.getAttribute("nodegraph")
                    if nodegraph_string:
                        impl = stdlib.getNodeGraph(nodegraph_string)
                        nodegraph_name = nodegraph_string
                    if nodegraph_name:
                        # Count usage of each nodegraph
                        if nodegraph_name in nodegraph_counts:
                            nodegraph_counts[nodegraph_name] += 1
                        else:
                            nodegraph_counts[nodegraph_name] = 1
                        # Track which nodedefs use this nodegraph
                        if nodegraph_name not in nodegraph_nodedefs:
                            nodegraph_nodedefs[nodegraph_name] = set()
                        nodegraph_nodedefs[nodegraph_name].add(ndstring)
                    if impl and impl.isA(mx.NodeGraph):
                        print("  mapped implementation:", impl.getName())

        impls = stdlib.getImplementations()
        print("*"*80)
        print("Mapped Node graph usage counts:")
        for ngname, count in nodegraph_counts.items():
            ndefs = nodegraph_nodedefs[ngname]
            for ndef in ndefs:
                print(f"  nodedef: {ndef}, nodegraph: {ngname}, count: {count}")

        print("-"*80 + "\n")


get_matching_definitions("ND_UsdUVTexture")
get_matching_definitions("ND_standard_surface_surfaceshader")


got node def: 2.2
* number of matching definitions: 2
matching definition: ND_UsdUVTexture


NameError: name 'nd_string' is not defined

In [ ]:
indirect_mapped_nodedefs = {}
direct_mapped_nodefs = {}

for ndef in stdlib.getNodeDefs():
    impl = ndef.getImplementation()
    if impl:
        if impl.isA(mx.NodeGraph) :
            direct_mapped_nodefs[ndef.getName()] = impl
            continue
        elif impl.hasAttribute("nodegraph"):
            indirect_mapped_nodedefs[ndef.getName()] = impl    

print("Directly mapped nodedefs count:", len(direct_mapped_nodefs))
print("Indirectly mapped nodedefs count:", len(indirect_mapped_nodedefs))
if len(indirect_mapped_nodedefs) > 0:
    print("Indirectly mapped nodedefs:")
    for ndef_name, impl in indirect_mapped_nodedefs.items():
        print(f"- nodedef: {ndef_name} -> implementation: {impl.getNamePath()}")

def getUnappedImplementation(find_nodedef_name, indirect_mapped_nodedefs):
    if find_nodedef_name in indirect_mapped_nodedefs:
        return indirect_mapped_nodedefs[find_nodedef_name]
    return None

impls = stdlib.getImplementations()
for ndef in stdlib.getNodeDefs():
    ndef_name = ndef.getName()
    unmapped_impl = getUnappedImplementation(ndef_name, indirect_mapped_nodedefs)
    if unmapped_impl:
        print(f"- found unmapped implementation {unmapped_impl.getNamePath()} for nodedef: {ndef_name}")

Directly mapped nodedefs count: 266
Indirectly mapped nodedefs count: 0
